In [138]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axis as axis

from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn import preprocessing
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf


sns.set(font_scale=1.25)
plt.style.use('seaborn-white')

<ipython-input-138-05971dcf0dd4>:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [139]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/dataframe1.csv'
df = pd.read_csv(file_path)
df.columns
df.drop("Date", axis = 1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Index(['Date', 'PNL', 'STD_A', 'STD_B', 'STD_C', 'STD_D', 'STD_E', 'STD_F',
       'STD_G', 'STD_Sum', 'STD_Total', 'STD_Diff', 'Open_Vol',
       'Perc_Change_Vol', 'Perc_Change_Price', 'Volume', 'Open_Int', 'Range',
       'High_Diff', 'High_Vol', 'High_PC_Price', 'High_PC_Vol'],
      dtype='object')

,PNL,STD_A,STD_B,STD_C,STD_D,STD_E,STD_F,STD_G,STD_Sum,STD_Total,...,Open_Vol,Perc_Change_Vol,Perc_Change_Price,Volume,Open_Int,Range,High_Diff,High_Vol,High_PC_Price,High_PC_Vol
0,-87826.0,86468.498575,10878.838940,12334.135120,2815.128735,34893.760498,1074.230010,124562.410578,273027.002457,208302.922302,...,37.12,0.87,1.46,443900.0,1886091.0,2.16,0,0,1.0,0.0
1,21268.0,91845.330007,94278.580341,9297.392074,3480.016427,34194.618568,21482.950192,121529.229587,376108.117196,230086.858942,...,38.27,5.64,2.70,704355.0,1881769.0,2.99,1,0,1.0,1.0
2,262982.0,94350.688837,21781.236370,8312.541272,642.888822,25922.699592,13779.058313,118436.921858,283226.035064,219589.530555,...,40.67,3.80,0.62,784946.0,1880445.0,1.25,0,0,0.0,1.0
3,-175431.0,114380.248146,16358.357476,19714.508052,2564.716941,34678.939436,9706.071673,118254.557267,315657.398990,228701.656093,...,36.96,7.12,3.59,1084446.0,1862150.0,3.88,0,0,1.0,1.0
4,44683.0,111122.888951,13866.942816,9517.710606,5498.290892,27595.620454,5983.076914,118929.949403,292514.480036,209646.713894,...,33.85,6.88,2.15,784520.0,1826195.0,2.57,0,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,-9212.0,159006.389726,16439.713255,5539.028234,6858.943954,17327.079776,3708.303937,246039.066881,454918.525762,336305.010981,...,36.86,0.38,0.36,979709.0,1918425.0,1.03,1,0,0.0,0.0
328,-155963.0,133102.043044,19114.299087,4241.599255,4516.749839,18645.720738,3701.869965,201477.474601,384799.756528,258774.805179,...,44.49,17.15,0.78,1170432.0,1913391.0,1.99,1,0,0.0,1.0
329,-130371.0,93783.326495,16130.048692,2284.753960,2403.311996,24296.427786,3608.884891,191750.646166,334257.399986,222386.049450,...,39.48,12.69,0.06,989697.0,1901975.0,1.22,1,0,0.0,1.0
330,35314.0,92013.490154,19061.987667,3007.815489,1289.283921,23963.530510,3569.635007,194111.365264,337017.108012,242245.463225,...,37.57,5.08,1.38,852063.0,1887131.0,1.37,0,0,0.0,1.0


divide the data into training and test data sets 

In [140]:
train_df = df[:258]  # First 258 rows as the training set (all of 2022)
test_df = df[258:]   # Remaining rows as the test set (2023 until May 1st)





In [141]:
c = len(df.axes[1])

df2 = train_df.copy()


column_names = df2.columns.values.tolist()

for col in column_names[1:]:
  corr_list= []
  corr = train_df[col].corr(train_df['PNL'])
  corr = np.absolute(corr)
  if corr < 0.2:
    train_df = train_df.drop(col,axis=1)

print(train_df.shape)
train_df

(258, 10)


,Date,PNL,STD_E,STD_F,STD_G,Open_Vol,Perc_Change_Price,Volume,Range,High_Vol
0,2023-05-01,-87826.0,34893.760498,1074.230010,124562.410578,37.12,1.46,443900.0,2.16,0
1,2023-04-28,21268.0,34194.618568,21482.950192,121529.229587,38.27,2.70,704355.0,2.99,0
2,2023-04-27,262982.0,25922.699592,13779.058313,118436.921858,40.67,0.62,784946.0,1.25,0
3,2023-04-26,-175431.0,34678.939436,9706.071673,118254.557267,36.96,3.59,1084446.0,3.88,0
4,2023-04-25,44683.0,27595.620454,5983.076914,118929.949403,33.85,2.15,784520.0,2.57,0
...,...,...,...,...,...,...,...,...,...,...
253,2022-04-26,-167594.0,13663.182845,17900.130835,110034.694004,52.86,1.33,685031.0,2.53,1
254,2022-04-25,170622.0,15900.755996,13461.507748,113509.334397,56.00,2.29,647830.0,3.19,1
255,2022-04-22,-79995.0,16575.373709,1576.639481,112510.731802,49.86,2.42,440249.0,2.95,1
256,2022-04-21,-138847.0,11167.993145,11568.394751,96018.980693,52.23,1.65,551943.0,2.35,1


In [142]:
df3  = train_df.drop('PNL', axis =1 )
df3 = df3.drop("Date", axis = 1)
column_names2 = df3.columns.values.tolist()
num_of_col = len(column_names2)
drop_list = []
for i in range(num_of_col):
  first = column_names2[i]
  j= i +1 

  for j in range(j,num_of_col):
  
    second =  column_names2[j]
    corr = df3[first].corr(df3[second])
    if corr > 0.75:
      drop_list.append(second)

drop_list = set(drop_list)
drop_list = list(drop_list)

for i in drop_list:

  train_df = train_df.drop(i,axis=1)

train_df

,Date,PNL,STD_E,STD_F,STD_G,Open_Vol,Perc_Change_Price,Volume,Range
0,2023-05-01,-87826.0,34893.760498,1074.230010,124562.410578,37.12,1.46,443900.0,2.16
1,2023-04-28,21268.0,34194.618568,21482.950192,121529.229587,38.27,2.70,704355.0,2.99
2,2023-04-27,262982.0,25922.699592,13779.058313,118436.921858,40.67,0.62,784946.0,1.25
3,2023-04-26,-175431.0,34678.939436,9706.071673,118254.557267,36.96,3.59,1084446.0,3.88
4,2023-04-25,44683.0,27595.620454,5983.076914,118929.949403,33.85,2.15,784520.0,2.57
...,...,...,...,...,...,...,...,...,...
253,2022-04-26,-167594.0,13663.182845,17900.130835,110034.694004,52.86,1.33,685031.0,2.53
254,2022-04-25,170622.0,15900.755996,13461.507748,113509.334397,56.00,2.29,647830.0,3.19
255,2022-04-22,-79995.0,16575.373709,1576.639481,112510.731802,49.86,2.42,440249.0,2.95
256,2022-04-21,-138847.0,11167.993145,11568.394751,96018.980693,52.23,1.65,551943.0,2.35


no colinearity exists 

In [143]:
train_df.describe()





,PNL,STD_E,STD_F,STD_G,Open_Vol,Perc_Change_Price,Volume,Range
count,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,2.580000e+02,258.000000
mean,-1406.178295,21985.156040,10657.883255,167391.628875,47.130271,1.739380,7.542844e+05,2.843605
std,246989.833721,7597.634159,9136.075379,84530.805722,6.476654,1.303185,1.830976e+05,1.086025
min,-777349.000000,4945.843560,875.540100,66382.835194,32.810000,0.000000,3.969590e+05,0.880000
25%,-172679.500000,16633.143280,4318.372154,95967.674650,42.437500,0.802500,6.495575e+05,2.152500
50%,-7784.000000,22106.215198,7735.365577,124408.910429,47.860000,1.465000,7.398335e+05,2.705000
75%,144092.000000,26273.064144,14249.869839,242073.430285,52.245000,2.417500,8.439288e+05,3.327500
max,851128.000000,39292.703565,65505.993487,393959.791481,60.480000,8.830000,1.752725e+06,11.210000


In [144]:
est = smf.ols('PNL ~ STD_E + STD_G + STD_F + Volume + Open_Vol + Perc_Change_Price + Range', train_df).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PNL   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     12.34
Date:                Wed, 17 May 2023   Prob (F-statistic):           1.51e-13
Time:                        03:10:46   Log-Likelihood:                -3530.9
No. Observations:                 258   AIC:                             7078.
Df Residuals:                     250   BIC:                             7106.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -2.321e+05    1.7e+05     -1.363      0.174   -5.67e+05    1.03e+05
STD_E                 0.7232      2.389      0.303      0.762      -3.982       5.428
STD_G                 0.7181      0.184      3.904      0.000       0.356       1.080
STD_F                 4.3049      1.546      2.785      0.006       1.261       7.349
Volume                0.1869      0.093      2.010      0.046       0.004       0.370
Open_Vol          -5644.7186   2828.450     -1.996      0.047   -1.12e+04     -74.092
Perc_Change_Price  1226.0066    1.5e+04      0.082      0.935   -2.82e+04    3.07e+04
Range              6.034e+04    2.1e+04      2.871      0.004    1.89e+04    1.02e+05
==============================================================================
Omnibus:                       13.712   Durbin-Watson:                   1.943
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.946
Skew:                          -0.385   Prob(JB):                     7.69e-05
Kurtosis:                       4.082   Cond. No.                     1.01e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

STD_E and Perc_Change_Price have P values greater than 0.05 so they are not significant and should be removed

In [155]:
est = smf.ols('PNL ~  STD_G + STD_F + Volume + Open_Vol + Range', train_df).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PNL   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     17.38
Date:                Wed, 17 May 2023   Prob (F-statistic):           8.89e-15
Time:                        03:12:48   Log-Likelihood:                -3531.0
No. Observations:                 258   AIC:                             7074.
Df Residuals:                     252   BIC:                             7095.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.024e+05   1.39e+05     -1.460      0.146   -4.75e+05    7.07e+04
STD_G          0.7287      0.180      4.052      0.000       0.375       1.083
STD_F          4.3819      1.520      2.882      0.004       1.388       7.376
Volume         0.1927      0.091      2.124      0.035       0.014       0.371
Open_Vol   -6077.5423   2462.692     -2.468      0.014   -1.09e+04   -1227.461
Range       6.095e+04   1.61e+04      3.789      0.000    2.93e+04    9.26e+04
==============================================================================
Omnibus:                       13.529   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.610
Skew:                          -0.382   Prob(JB):                     9.10e-05
Kurtosis:                       4.072   Cond. No.                     8.22e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.22e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [159]:

pred = est.predict(test_df)
test_df['Predict'] = pred
dff = test_df[['PNL', 'Predict']]

RMSE = ((dff.PNL - dff.Predict) ** 2).mean() ** 0.5


<ipython-input-159-1e62ad39174e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Predict'] = pred


In [160]:
RMSE

351518.84053580597

RMSE of 351,518 is massive and this is supported by the fact that the R-squared value of the regression is only 0.25; thus the model does not fit the data well 

Next I will verify by Subset evaluation

In [147]:
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools

sns.set(font_scale=1.25)

In [148]:
y= train_df.PNL
X = train_df.drop(['PNL','Date'], axis=1)
n2 = X.shape[1]
n2
X

7

,STD_E,STD_F,STD_G,Open_Vol,Perc_Change_Price,Volume,Range
0,34893.760498,1074.230010,124562.410578,37.12,1.46,443900.0,2.16
1,34194.618568,21482.950192,121529.229587,38.27,2.70,704355.0,2.99
2,25922.699592,13779.058313,118436.921858,40.67,0.62,784946.0,1.25
3,34678.939436,9706.071673,118254.557267,36.96,3.59,1084446.0,3.88
4,27595.620454,5983.076914,118929.949403,33.85,2.15,784520.0,2.57
...,...,...,...,...,...,...,...
253,13663.182845,17900.130835,110034.694004,52.86,1.33,685031.0,2.53
254,15900.755996,13461.507748,113509.334397,56.00,2.29,647830.0,3.19
255,16575.373709,1576.639481,112510.731802,49.86,2.42,440249.0,2.95
256,11167.993145,11568.394751,96018.980693,52.23,1.65,551943.0,2.35


In [149]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    RMSE = (((regr.predict(X[list(feature_set)]) - y) ** 2).mean()) ** 0.5
    return {"model":regr, "RMSE":RMSE }

In [150]:
from statsmodels import regression
def getBest(k):
    results = []
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    # Choose the model with the lowest RMSE
    best_model = models.loc[models['RMSE'].argmin()]
    print("Processed ", models.shape[0], "models on", k, "predictors.")
    # Return the best model
    return best_model

In [151]:
models = pd.DataFrame(columns=["RMSE", "model"])
for i in range(1,n2+1):
    models.loc[i] = getBest(i)

Processed  7 models on 1 predictors.
Processed  21 models on 2 predictors.
Processed  35 models on 3 predictors.
Processed  35 models on 4 predictors.
Processed  21 models on 5 predictors.
Processed  7 models on 6 predictors.
Processed  1 models on 7 predictors.


In [152]:
models

,RMSE,model
1,244104.818878,<statsmodels.regression.linear_model.Regressio...
2,227210.343181,<statsmodels.regression.linear_model.Regressio...
3,218286.628495,<statsmodels.regression.linear_model.Regressio...
4,214716.749134,<statsmodels.regression.linear_model.Regressio...
5,213465.348538,<statsmodels.regression.linear_model.Regressio...
6,213318.813896,<statsmodels.regression.linear_model.Regressio...
7,213313.922038,<statsmodels.regression.linear_model.Regressio...


In [153]:
models.apply(lambda row: row[1].rsquared, axis=1)

1    0.019456
2    0.150486
3    0.215905
4    0.241342
5    0.250159
6    0.251188
7    0.251223
dtype: float64

In [158]:
best_model = models.loc[7,"model"]
best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    PNL   R-squared (uncentered):                   0.251
Model:                            OLS   Adj. R-squared (uncentered):              0.230
Method:                 Least Squares   F-statistic:                              12.03
Date:                Wed, 17 May 2023   Prob (F-statistic):                    3.14e-13
Time:                        03:14:26   Log-Likelihood:                         -3531.9
No. Observations:                 258   AIC:                                      7078.
Df Residuals:                     251   BIC:                                      7103.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
STD_E                -1.1532      1.955     -0.590      0.556      -5.004       2.698
STD_F                 4.0945      1.540      2.658      0.008       1.061       7.128
STD_G                 0.6514      0.178      3.668      0.000       0.302       1.001
Open_Vol          -9155.1668   1169.104     -7.831      0.000   -1.15e+04   -6852.662
Perc_Change_Price  1607.9587    1.5e+04      0.107      0.915   -2.79e+04    3.11e+04
Volume                0.1669      0.092      1.814      0.071      -0.014       0.348
Range              6.172e+04    2.1e+04      2.936      0.004    2.03e+04    1.03e+05
==============================================================================
Omnibus:                       11.827   Durbin-Watson:                   1.913
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               16.114
Skew:                          -0.339   Prob(JB):                     0.000317
Kurtosis:                       4.019   Cond. No.                     1.40e+06
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

This shows the best model is the predictors in the last regression , but again with an r-squared value of .25, the model does not accurately describe the data.